# Graph structure modelling 

- functions permitting to model the 1) graph taxonomy and ensure 2) graph regularizarion
- from https://github.com/PlusRoss/GRCN/tree/master 

In [12]:
# libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
import matplotlib
import random
import time

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import kneighbors_graph

import networkx as nx

# Building graph from the data points

In [22]:
data = pd.read_csv("train2s.csv", header = None)
data = pd.read_csv("test3s.csv", header = None)

In [23]:
data

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,12.6875,25.0625,14.2500,9.3125,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0
1,3.0000,33.1250,32.6250,10.6250,5.1875,6.6875,7.1875,6.5000,0.0625,0.0625,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0
2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1250,0.2500,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0
3,0.0000,0.0625,0.0625,4.4375,8.7500,22.3750,68.3750,65.1875,60.5000,34.6250,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0
4,0.0000,0.0000,7.6875,24.1875,26.6875,24.4375,56.3125,16.8750,3.3750,4.9375,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562,0.0000,0.0000,0.0000,0.0000,0.0000,0.6875,18.8125,72.3125,54.6875,1.7500,...,0.0,0.0,8.8125,26.0625,1.6875,0.0000,0.0000,0.0000,0.0000,0.0
2563,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,6.0000,64.5000,88.5000,14.8125,...,0.0,0.0,0.0000,0.2500,3.5000,1.8125,0.8125,0.0000,0.0000,0.0
2564,0.1250,23.6250,9.0625,0.3125,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0
2565,0.0625,5.3750,60.5000,19.3750,0.6250,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0


In [9]:
# labels_train  # list of labels


# Loading of datasets

spectr_train_list = pd.read_csv("spectr_train_list.txt", index_col = 0).iloc[:,0].values.tolist()
spectr_test_list = pd.read_csv("spectr_test_list.txt", index_col = 0).iloc[:,0].values.tolist()
# spectr_valid_list = pd.read_csv("spectr_valid_list.txt", index_col = 0).iloc[:,0].values.tolist()

# labels_train, labels_test
# spectrograms_train, spectrograms_test

labels_train = []

for path in spectr_train_list:
    labels_train.append(path.split("/")[1])
    
labels_test = []

for path in spectr_test_list:
    labels_test.append(path.split("/")[1])

# BASIC kNN method

In [10]:
def adjacencyMatrixUsingMnearestNeighbors(X, M = 5):
    
    knn = NearestNeighbors(n_neighbors=M)
    knn.fit(X)
    #dist_indx_arr = knn.kneighbors(X, return_distance=True, n_neighbors = M)
    
    A = kneighbors_graph(X, n_neighbors=(M-1), p=2, mode='connectivity', include_self=False) 
    A = A.toarray() # for some reason, is not symmetric
    A = np.maximum( A, A.T )
    
    return A#, dist_indx_arr # dist_indx_arr will be useful while connecting the graph


# Can be used if needed - maybe the graph does not need to be connected?
#
# Function implements trivial method of connecting the graph
#
# A - adjacency matrix
# returns: adjacency matrix with additional connections
def connectTheGraph(A):
    # trivial case of adding an edge:
    W = A
    G = nx.from_numpy_array(W)
    graphs = list(nx.connected_components(G))
    for i in range(len(graphs)-1):

        for g in graphs[i]:
            for f in graphs[i+1]:
                W[f,g] = 1
                W[g,f] = 1
                break
            break
    return W

In [24]:
A = adjacencyMatrixUsingMnearestNeighbors(data, M=5) # it needs 1-10s (seems increase linearly with n of observations)

In [25]:
G = nx.from_numpy_array(A)

In [27]:
nx.is_connected(G)
# nx.draw_kamada_kawai(G) # do not draw - it is too computationally demanding

True

# GRCN method - todo

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SGConv, GATConv, knn_graph
from sklearn.neighbors import kneighbors_graph
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np
from model_utils import GCNConv_diag, GCNConv_dense, EOS
# import torch_sparse as ts # it poses problems

In [26]:
!pip install torch_sparse

In [9]:
class Model(torch.nn.Module):

    def __init__(self, num_nodes, num_features, num_classes, device, args):
        super(Model, self).__init__()
        self.num_nodes = num_nodes
        self.num_features = num_features
        self.graph_nhid = int(args.hid_graph.split(":")[0])
        self.graph_nhid2 = int(args.hid_graph.split(":")[1])
        self.nhid = args.nhid
        self.conv1 = GCNConv_dense(num_features, self.nhid)
        self.conv2 = GCNConv_dense(self.nhid, num_classes)
        if args.layertype == "dense":
            self.conv_graph = GCNConv_dense(num_features, self.graph_nhid)
            self.conv_graph2 = GCNConv_dense(self.graph_nhid, self.graph_nhid2)
        elif args.layertype == "diag":
            self.conv_graph = GCNConv_diag(num_features, device)
            self.conv_graph2 = GCNConv_diag(num_features, device)
        else:
            exit("wrong layer type")
        self.F = args.F
        self.F_graph = args.F_graph
        self.dropout = args.dropout
        self.K = args.compl_param.split(":")[0]
        self.mask = None
        self.Adj_new = None
        self._normalize = args.normalize
        self.device = device
        self.reduce = args.reduce
        self.sparse = args.sparse
        self.norm_mode = "sym"
        self.topindex = None

    def init_para(self):
        self.conv1.init_para()
        self.conv2.init_para()
        self.conv_graph.init_para()
        self.conv_graph2.init_para()

    def graph_parameters(self):
        return list(self.conv_graph.parameters()) + list(self.conv_graph2.parameters())

    def base_parameters(self):
        return list(self.conv1.parameters()) + list(self.conv2.parameters())

    def cal_similarity_graph(self, node_embeddings):
        # similarity_graph = torch.mm(node_embeddings, node_embeddings.t())
        similarity_graph = torch.mm(node_embeddings[:, :int(self.num_features/2)], node_embeddings[:, :int(self.num_features/2)].t())
        similarity_graph += torch.mm(node_embeddings[:, int(self.num_features/2):], node_embeddings[:, int(self.num_features/2):].t())
        # node_embeddings_top = node_embeddings[self.topindex]
        # similarity_graph = node_embeddings.unsqueeze(1) * node_embeddings_top
        # similarity_graph = torch.sum(similarity_graph, dim=-1)
        return similarity_graph

    def normalize(self, adj, mode="sym" ,sparse=False):
        if not sparse:
            if mode == "sym":
                inv_sqrt_degree = 1. / (torch.sqrt(adj.sum(dim=1, keepdim=False)) + EOS)
                return inv_sqrt_degree[:, None] * adj * inv_sqrt_degree[None, :]
            elif mode == "row":
                inv_degree = 1. / (adj.sum(dim=1, keepdim=False) + EOS)
                return inv_degree[:, None] * adj
            else:
                exit("wrong norm mode")
        else:
            adj = adj.coalesce()
            if mode == "sym":
                inv_sqrt_degree = 1. / (torch.sqrt(torch.sparse.sum(adj, dim=1).values()) + EOS)
                D_value = inv_sqrt_degree[adj.indices()[0]] * inv_sqrt_degree[adj.indices()[1]]
            elif mode == "row":
                inv_degree = 1. / (torch.sparse.sum(adj, dim=1).values() + EOS)
                D_value = inv_degree[adj.indices()[0]]
            else:
                exit("wrong norm mode")
            new_values = adj.values() * D_value
            return torch.sparse.FloatTensor(adj.indices(), new_values, adj.size()).to(self.device)

    def _sparse_graph(self, raw_graph, K, sparse):
        if self.reduce == "knn":
            if self.topindex is None:
                values, indices = raw_graph.topk(k=int(K), dim=-1)
                self.topindex = indices
                assert torch.max(indices) < raw_graph.shape[1]
            else:
                indices = self.topindex
                values = raw_graph[torch.arange(raw_graph.shape[0]).view(-1,1), indices]
            assert torch.sum(torch.isnan(values)) == 0
            if not sparse:
                self.mask = torch.zeros(raw_graph.shape).to(self.device)
                self.mask[torch.arange(raw_graph.shape[0]).view(-1,1), indices] = 1.
                self.mask[indices, torch.arange(raw_graph.shape[1]).view(-1,1)] = 1.
            else:
                inds = torch.stack([torch.arange(raw_graph.shape[0]).view(-1,1).expand(-1,int(K)).contiguous().view(1,-1)[0].to(self.device),
                                     indices.view(1,-1)[0]])
                inds = torch.cat([inds, torch.stack([inds[1], inds[0]])], dim=1)
                values = torch.cat([values.view(1,-1)[0], values.view(1,-1)[0]])
                return inds, values
        else:
            exit("wrong sparsification method")
        self.mask.requires_grad = False
        sparse_graph = raw_graph * self.mask
        return sparse_graph

    def _node_embeddings(self, input, Adj, sparse=False):
        norm_Adj = self.normalize(Adj, self.norm_mode, sparse)
        if self.F_graph != "identity":
            node_embeddings = self.F_graph(self.conv_graph(input, norm_Adj, sparse))
            node_embeddings = self.conv_graph2(node_embeddings, norm_Adj, sparse)
        else:
            node_embeddings = self.conv_graph(input, norm_Adj, sparse)
            node_embeddings = self.conv_graph2(node_embeddings, norm_Adj, sparse)
        if self._normalize:
            node_embeddings = F.normalize(node_embeddings, dim=1, p=2)
        return node_embeddings

    def forward(self, input, Adj):
        Adj.requires_grad = False
        node_embeddings = self._node_embeddings(input, Adj, self.sparse)
        Adj_new = self.cal_similarity_graph(node_embeddings)

        if not self.sparse:
            Adj_new = self._sparse_graph(Adj_new, self.K, self.sparse)
            Adj_new = self.normalize(Adj + Adj_new, self.norm_mode)
        else:
            Adj_new_indices, Adj_new_values = self._sparse_graph(Adj_new, self.K, self.sparse)
            new_inds = torch.cat([Adj.indices(), Adj_new_indices], dim=1)
            new_values = torch.cat([Adj.values(), Adj_new_values])
            Adj_new = torch.sparse.FloatTensor(new_inds, new_values, Adj.size()).to(self.device)
            Adj_new = self.normalize(Adj_new, self.norm_mode, self.sparse)

        x = self.conv1(input, Adj_new, self.sparse)
        x = F.dropout(self.F(x), training=self.training, p=self.dropout)
        x = self.conv2(x, Adj_new, self.sparse)

        return F.log_softmax(x, dim=1)
